In [1]:
path = "/home/vatsal/NWM/CasCast/latent_data/earthformer_example/test_2h/vil-2019-SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5-0-2.npy"
import h5py
import numpy as np
import matplotlib.pyplot as plt

import os
data = np.load(path)
print(data.shape)

(12, 4, 48, 48)


In [1]:
import os
import pickle
from pprint import pprint
with open("/home/vatsal/NWM/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/experiments/pret3/scores/test_results_epoch_0.pkl", "rb") as f:
    data = pickle.load(f)
    pprint(data)

{16: {'bias': 2.674252510070801,
      'csi': 0.7783332467079163,
      'pod': 0.9337869882583618,
      'sucr': 0.8237987756729126},
 74: {'bias': 2.1175405979156494,
      'csi': 0.7116419076919556,
      'pod': 0.8351284861564636,
      'sucr': 0.8279649615287781},
 133: {'bias': 1.911167025566101,
       'csi': 0.46627241373062134,
       'pod': 0.6227177381515503,
       'sucr': 0.6498549580574036},
 160: {'bias': 1.1911898851394653,
       'csi': 0.35356423258781433,
       'pod': 0.4588181972503662,
       'sucr': 0.6064914464950562},
 181: {'bias': 0.9405474066734314,
       'csi': 0.3053286671638489,
       'pod': 0.3911496102809906,
       'sucr': 0.5818714499473572},
 219: {'bias': 0.6369980573654175,
       'csi': 0.1833953708410263,
       'pod': 0.2407078593969345,
       'sucr': 0.43510666489601135},
 'avg': {'bias': 1.5786159137884777,
         'csi': 0.4664226397871971,
         'pod': 0.5803848132491112,
         'sucr': 0.6541813760995865},
 'mae': tensor(0.0311, dev

In [ ]:
path3 = "/home/vatsal/NWM/earth-forecasting-transformer/datasets/sevir/data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0901_1231.h5"
data3 = h5py.File(path3, 'r')
data3 = data3['vil'][:] 
print(data3.shape)


(1900, 384, 384, 49)


In [5]:
path4 = "/home/vatsal/NWM/earth-forecasting-transformer/datasets/sevir/data/vil/2018/SEVIR_VIL_STORMEVENTS_2018_0101_0630.h5"
data4 = h5py.File(path4, 'r')
data4 = data4['vil'][:]
print(data4.shape)

(793, 384, 384, 49)


In [6]:
import os
import fnmatch

# Ignore patterns (directories and file patterns)
ignore_dirs = ['wandb', '__pycache__']
ignore_patterns = [
    '*.nc', '*.pyc', '*.h5', '*.pkl', '*.pt', '*.pth', '*.npy', '*.npz',
    '*_env', '*.ckpt', '*.png', '*.jpeg', '*.jpg', '*.gif'
]

# Size threshold in bytes (5 MB)
size_threshold = 5 * 1024 * 1024

def matches_ignore(path):
    # Check if any ignored dir is part of the path
    for d in ignore_dirs:
        if d in path.split(os.sep):
            return True
    # Check file patterns
    for pattern in ignore_patterns:
        if fnmatch.fnmatch(os.path.basename(path), pattern):
            return True
    return False

def find_large_files(root='.'):
    large_files = []
    for dirpath, dirnames, filenames in os.walk(root):
        # Skip ignored directories early
        dirnames[:] = [d for d in dirnames if d not in ignore_dirs]

        for f in filenames:
            filepath = os.path.join(dirpath, f)
            if matches_ignore(filepath):
                continue
            try:
                size = os.path.getsize(filepath)
                if size > size_threshold:
                    large_files.append((filepath, size))
            except OSError:
                pass
    return large_files

if __name__ == "__main__":
    results = find_large_files()
    for path, size in results:
        print(f"{path} - {size / (1024*1024):.2f} MB")


./Graph_gen_cast/2312.15796v2.pdf - 39.02 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so - 421.39 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/lib/libtorch_cuda.so - 860.84 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/lib/libtorch_python.so - 28.17 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/lib/libtorch_cuda_linalg.so - 91.82 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/bin/protoc - 5.08 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/torch/bin/protoc-3.13.0.0 - 5.08 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/netCDF4/_netCDF4.cpython-312-x86_64-linux-gnu.so - 12.25 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/pyogrio.libs/libgdal-44263852.so.35.3.9.1 - 49.88 MB
./FourcastNet/FourCastNet/venv/lib/python3.12/site-packages/rasterio.libs/libgdal-rasterio-40212d0b.so.35.3.9.3 - 21.68 MB
./FourcastNet/